## Gathering Data 


Publically available data containing movie/tv shows genres, rating, and year of release has already been collected by various reputable sources. We chose to investigate the IMDb Datasets (https://www.imdb.com/interfaces/). These datasets have movie data going back all the way to 1874              . The datasets also contain a wide range of potentially interesting data including: if it's an Adult movie, the production year, other(potentially foriegn) titles , names of directors, names of writers, etc. Another big positive to working with these datasets is that they are completely free to the public.  


Meanwhile with other media datasets, while they are just as extensive as IMDb, they would require a payment to get the complete dataset and not just a sample. For instance, another database we were considering using was The Numbers (https://www.the-numbers.com/box-office-records/domestic/all-movies/cumulative/all-time/2501). However, we realised to get the data we required we would have to pay $475. 


## Data Processing


There times when the data you recieve from the databases are not in the correct format. Or, in a format that is difficult for the system to comprehend/handle. 

We will be using three datasets from IMDb. We must use three datasets instead of one since each dataset has relevant data to our question. However, there isn;t an only made dataset that has all the information. As a result, we must upload all three databases, get the relevant infomation for each, and then combine the relavent data into one dataframe.

As mentioned in the Gathering Data section, one positive to IMDb Datasaets is that we would have a large dataset to analyze and explore. However, it is also a negative. Because it is so large (a total of 710 MB!!), it is impossible for most laptops/computers to process the data all at once. As a result, it must be condensed to only contain data we think may be relavent in future analysis and must be proccessed in chunks. 

NOTE: This is a common problem in data science and would most likely need to be addressed when you do data science analysis in the future.


In [1]:
import pandas as pd

#dictates the number of rows each data frame should have
chunksize = 100000

# the list that contains all the dataframes
list_of_dataframes = []

#Seperates title.basics.tsv into chunks with 100000 rows with the header of row 0.
#The columns of the dataframes would be tconst, originalTitle, startYear (publication year), 
#and genres data. each chunck is added to list_of_dataframes
for df in pd.read_csv("title.basics.tsv", sep='\t', chunksize=chunksize, 
                       header=0, usecols=['tconst', 'primaryTitle', 'genres', 'startYear', 'endYear']):
      list_of_dataframes.append(df)
        
#concats all the chunck dataframes to make one combined        
data1 = pd.concat(list_of_dataframes)

/Users/aaddeh/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
data1.head()

,tconst,primaryTitle,startYear,endYear,genres
0,tt0000001,Carmencita,1894,\N,"Documentary,Short"
1,tt0000002,Le clown et ses chiens,1892,\N,"Animation,Short"
2,tt0000003,Pauvre Pierrot,1892,\N,"Animation,Comedy,Romance"
3,tt0000004,Un bon bock,1892,\N,"Animation,Short"
4,tt0000005,Blacksmith Scene,1893,\N,"Comedy,Short"


Each movie instance/row has the following features/attributes: 
* tconst  - IMDb's alphanumeric unique identifier of the movie title
* primaryTitle  – the more popular movie title / the movietitle used by the filmmakers on promotional     materials at the point of release
* startYear  – represents the release year of a movie title.  In the case of TV Series, it is the series start year
* endYear  – TV Series end year. ‘\N’ for all other title types
* genres  – includes up to three genres associated with the title

All of them are object types.





In [3]:
#makes a dataframe from the data in the title.ratings.tsv database
#averageRating is the average IMDb rating , numVotes is the number of IMDd
#votes the movie recieved
data2= pd.read_csv("title.ratings.tsv", sep='\t')
data2.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1718
1,tt0000002,6.0,211
2,tt0000003,6.5,1480
3,tt0000004,6.1,124
4,tt0000005,6.2,2283


Each movie instance/row has the following features/attributes: 
* tconst - IMDb's alphanumeric unique identifier of the movie title (same as dataframe above)
* averageRating  – weighted average of all the individual user ratings
* numVotes  - number of votes the title has received

All of them are object types.


In [4]:
#makes a dataframe from a portion of data in the data.tsv database
#it only collects the columns of titleID and region and puts that in a dataframe
data3= pd.read_csv("data.tsv", sep='\t', usecols=['titleId', 'region'])

# gets all the rows with the region being US 
data3 = data3[data3['region'] == 'US']

#removes the region column since it is no longer needed
data3=data3.drop(columns=['region'])

data3.head()

,titleId
5,tt0000001
14,tt0000002
35,tt0000005
39,tt0000005
40,tt0000005


Each movie instance/row has the following features/attributes: 
* titleID (object) - IMDb's alphanumeric unique identifier of the movie title (same as dataframe above)



#### Combining the dataframes

In [33]:
#merges (through inner join) data1 and data2 based on the tconst values
#and makes it a dataframe named partial_data_table
partial_table= pd.merge(data1, data2, on='tconst')
partial_table.head()

,tconst,primaryTitle,startYear,endYear,genres,averageRating,numVotes
0,tt0000001,Carmencita,1894,\N,"Documentary,Short",5.7,1718
1,tt0000002,Le clown et ses chiens,1892,\N,"Animation,Short",6.0,211
2,tt0000003,Pauvre Pierrot,1892,\N,"Animation,Comedy,Romance",6.5,1480
3,tt0000004,Un bon bock,1892,\N,"Animation,Short",6.1,124
4,tt0000005,Blacksmith Scene,1893,\N,"Comedy,Short",6.2,2283


##### Why Inner join? (COMPLETE)

In [34]:
#filters partial_data_table based on weither the row's title constant can be found in data3
#Full_table has all the information (title, genres, averateRating, etc) for only the US productions
full_table=partial_table.loc[partial_table["tconst"].isin(data3["titleId"])] 
full_table.head()


,titleId,primaryTitle,startYear,endYear,genres,averageRating,numVotes
0,tt0000001,Carmencita,1894,\N,"Documentary,Short",5.7,1718
1,tt0000002,Le clown et ses chiens,1892,\N,"Animation,Short",6.0,211
4,tt0000005,Blacksmith Scene,1893,\N,"Comedy,Short",6.2,2283
5,tt0000006,Chinese Opium Den,1894,\N,Short,5.1,127
6,tt0000007,Corbett and Courtney Before the Kinetograph,1894,\N,"Short,Sport",5.5,696


#### Processing unavailable data in the Dataframe

It is very important the data within the dataframe that is unavailable is categorized properly and uniformly.

There is no uniform rule on how different databases will represent missing data. Some databases put '-', others write 'NONE', or an empty string. There is an infinite amount of possibilities that databases could use. However, what is important from the data science perspective is how will Pandas interpret those values and having a concesus within the community. 

Inorder to Pandas (and the community) to understand that a value is missing, that cell must have one of these values:

* numpy.NaN (numpy): A number  value is not available in a number column
* numpy.nan (numpy): Same as the above (either one could be use for a missing number)
* pd.NaT (pandas):A time value in a time column is not available
* pd.NA (pandas): A general type of value is Not Available
* None (python): similar to pd.NA

Note: Data has an underlying meaning. If something was meant to be number but turned out not to be, that could be potentially different from it not being available. 

According to the IMBd website, their databases use ‘\N’ to denote that a particular field is missing or null for that title/name. These must be converted to an appropriate value from the list above. 


In [35]:
import numpy 

#replaces all instance of missing values in the full_table 
#dataframe as replace it with a pd Null or numpy Null value
full_table['primaryTitle'].replace({'\\N': pd.NA}, inplace=True)
full_table['genres'].replace({'\\N': pd.NA}, inplace=True)
full_table['startYear'].replace({'\\N': pd.NaT}, inplace=True) 
full_table['endYear'].replace({'\\N': pd.NaT}, inplace=True)
full_table['averageRating'].replace({'\\N':numpy.NaN}, inplace=True)
full_table['numVotes'].replace({'\\N':numpy.NaN}, inplace=True)

/Users/aaddeh/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [36]:
#gets the rows where the endYear is a null value
full_table=full_table[full_table['endYear'].isna()]

#removes the endYear column since it is no longer needed
full_table=full_table.drop(columns=['endYear'])

#renames two of the columns to more appropriate names
full_table=full_table.rename(columns={"primaryTitle": "title", "startYear": "releaseYear"})

full_table.head()

,titleId,title,releaseYear,genres,averageRating,numVotes
0,tt0000001,Carmencita,1894,"Documentary,Short",5.7,1718
1,tt0000002,Le clown et ses chiens,1892,"Animation,Short",6.0,211
4,tt0000005,Blacksmith Scene,1893,"Comedy,Short",6.2,2283
5,tt0000006,Chinese Opium Den,1894,Short,5.1,127
6,tt0000007,Corbett and Courtney Before the Kinetograph,1894,"Short,Sport",5.5,696


We are removing the rows that don't have a null value in the endYear column because, as noted on the IMDb api website, when an endYear is provided that means it is a tv series. Not a movie. Since we only care about movies, the information about TV series are irrevalant and should be delected to prevent possible future errors 